# Setttings

In [1]:
import numpy as np
import tensorflow as tf

/Users/mdeblaauw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read in data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../Tensorflow-applications/MNIST_data//", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../Tensorflow-applications/MNIST_data//t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-applications/MNIST_data//t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print(mnist.train.images.shape)
print(mnist.validation.images.shape)
print(mnist.test.images.shape)

(55000, 784)
(5000, 784)
(10000, 784)


In [5]:
mnist.train.labels[:,:].shape

(55000, 10)

# Create graph

### Settings

In [51]:
epsilon = 1e-7
learning_rate = 0.001
batch_size = 50
epochs = 10
rounds = 3

### Input and output tensors

In [52]:
X = tf.placeholder(tf.float32, [None, 784])
X_reshape = tf.reshape(X, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

## Squash function

In [53]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

### Layers

#### Convolutional

In [54]:
conv_filter_shape = [9, 9, 1, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W1')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B1')
out_layer = tf.nn.conv2d(X_reshape, weights, [1,1,1,1], padding = 'VALID')
out_layer_bias = tf.add(out_layer, bias)
layer1 = tf.nn.relu(out_layer_bias)

#### PrimaryCaps

In [55]:
conv_filter_shape = [9, 9, 256, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W2')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B2')
out_layer = tf.nn.conv2d(layer1, weights, [1,2,2,1], padding = 'VALID')
layer2 = tf.add(out_layer, bias)

In [56]:
caps1_raw = tf.reshape(layer2, [-1, 6*6*32, 8], name='caps1_raw')
caps1_output = squash(caps1_raw, axis=-1)

In [57]:
caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,10,1,1], name = 'caps1_out_tiled')

In [58]:
weight_matrix = tf.Variable(tf.truncated_normal([1, 1152, 10, 16, 8], stddev=0.1), name='W_matrix')
batch = tf.shape(X)[0]
weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')

In [59]:
caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')

### Routing by agreement

In [60]:
raw_weights = tf.zeros([batch, 1152, 10, 1, 1], name = 'raw_weights')

In [61]:
for i in range(rounds):
    routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
    weighted_predictions = tf.multiply(routing_weights, caps2_predicted, name = 'weighted_predictions' + str(i))
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
    caps2_output = squash(weighted_sum, axis=-2)
    
    caps2_output_tiled = tf.tile(caps2_output, [1, 1152, 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
    agreement = tf.matmul(caps2_predicted, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
    raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))

In [62]:
caps2_output

<tf.Tensor 'Mul_16:0' shape=(?, 1, 10, 16, 1) dtype=float32>

In [63]:
squared_norm_caps2 = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = True, name='caps2_norm')
caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

In [64]:
caps2_activation

<tf.Tensor 'caps2_activation_4:0' shape=(?, 1, 10, 1, 1) dtype=float32>

In [65]:
y_estimate = tf.squeeze(caps2_activation, axis=[1,3,4], name = 'y_pred')

In [66]:
y_pred = tf.nn.softmax(y_estimate, axis=1)

### Los function

In [67]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_estimate, labels=y))
optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

### Make predictions

In [68]:
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Init and saver

In [70]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

## Setup training

In [72]:
restore_checkpoint = True
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"

with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init_op.run()

    total_batch = int(len(mnist.train.images)/batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            a, c= sess.run([optimiser, cross_entropy], feed_dict={X: batch_x, y: batch_y})
            avg_cost += c/total_batch
            print('batch:', i)
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        train_acc = sess.run(accuracy, feed_dict={X: mnist.train.images, y: mnist.train.labels})
        test_acc = sess.run(accuracy, feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print('train_acc:', train_acc)
        print('val_acc:', val_acc)
        
        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15


KeyboardInterrupt: 